In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.visualize as visualize
import thor.analyze as analyze

notebook_name = "cpol_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2005-11-13T14:00:00"
end = "2005-11-13T16:00:00"

output_directory = base_local / "runs/cpol_demo_geographic"
options_directory = output_directory / "options"

if output_directory.exists():
    shutil.rmtree(output_directory)

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
cpol_options = data.aura.cpol_data_options(
    start=start, end=end, converted_options=converted_options
)
# Restrict the ERA5 data to a smaller region containing the CPOL radar
lon_range = [129, 133]
lat_range = [-14, -10]
era5_pl_options = data.era5.data_options(
    start=start, end=end, latitude_range=lat_range, longitude_range=lon_range
)
era5_sl_options = data.era5.data_options(
    start=start,
    end=end,
    data_format="single-levels",
    latitude_range=lat_range,
    longitude_range=lon_range,
)
data_options = option.consolidate_options(
    [cpol_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory)

altitude = list(np.arange(0, 20e3 + 500, 500))
altitude = [float(alt) for alt in altitude]
grid_options = grid.create_options(name="geographic", altitude=altitude)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)

# Create the track_options dictionary
track_options = option.mcs(dataset="cpol")
option.save_track_options(track_options, options_directory)

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
visualize_options["middle_echo"] = visualize.option.runtime_options(
    "middle_echo", save=True, style="presentation", figure_types=["mask"]
)
# visualize.option.save_display_options(visualize_options, options_directory)
visualize_options = None

2024-10-05 15:11:01,457 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/data.yml
2024-10-05 15:11:01,470 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-05 15:11:01,471 - thor.grid - WARNING - Coordinates not specified. Will attempt to infer from input.
2024-10-05 15:11:01,472 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/grid.yml
2024-10-05 15:11:01,482 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/track.yml


In [3]:
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory,
    parallel=False
)

2024-10-05 15:11:03,104 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol_demo_geographic.
2024-10-05 15:11:03,105 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-05 15:11:03,288 - thor.track - INFO - Processing 2005-11-13T14:00:09.
2024-10-05 15:11:03,290 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T14:00:09.
2024-10-05 15:11:03,291 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.140000.nc
2024-10-05 15:11:03,326 - thor.data.aura - INFO - Creating new geographic grid with spacing 0.025 m, 0.025 m.
2024-10-05 15:11:05,340 - thor.track - INFO - Processing hierarchy level 0.
2024-10-05 15:11:05,341 - thor.track - INFO - Tracking cell.
2024-10-05 15:11:13,299 - thor.match.match - INFO - Matching cell objects.
2024-10-05 15:11:13,301 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-10-05 15:11:13,306 - thor.track - INFO - Tracking middle_echo.
2024-1

# Cartesian Coordinates

In [40]:
output_directory = base_local / "runs/cpol_demo_cartesian"
options_directory = output_directory / "options"

if output_directory.exists():
    shutil.rmtree(output_directory)

grid_options = grid.create_options(name="cartesian", regrid=False, altitude=altitude)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)
data.option.save_data_options(data_options, options_directory)
option.save_track_options(track_options, options_directory)
# visualize.option.save_display_options(visualize_options, options_directory)

times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory,
)

2024-10-05 00:18:49,677 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-05 00:18:49,679 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/grid.yml
2024-10-05 00:18:49,691 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/data.yml
2024-10-05 00:18:49,735 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/track.yml
2024-10-05 00:18:49,758 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol_demo_cartesian.
2024-10-05 00:18:49,759 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-05 00:18:49,840 - thor.track - INFO - Processing 2005-11-13T12:00:09.
2024-10-05 00:18:49,842 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T12:00:09.
2024-10-05 00:18:49,843 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.120000.nc
20

KeyboardInterrupt: 

# Analysis

In [4]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_directory)
analyze.mcs.quality_control(output_directory, analysis_options)
analyze.mcs.classify_all(output_directory)

2024-10-05 15:11:28,019 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/velocities.csv
2024-10-05 15:11:28,022 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/velocities.yml
2024-10-05 15:11:28,137 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/quality.csv
2024-10-05 15:11:28,139 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/quality.yml
2024-10-05 15:11:28,192 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/classification.csv
2024-10-05 15:11:28,194 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_geographic/analysis/classification.yml


,,stratiform_offset,inflow,relative_stratiform_offset,tilt,propagation
time,universal_id,,,,,
2005-11-13 14:10:23,1,trailing,left,left,shear-perpendicular,up-shear
2005-11-13 14:20:09,1,trailing,left,left,up-shear,up-shear
2005-11-13 14:30:09,1,trailing,left,left,up-shear,up-shear
2005-11-13 14:40:09,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 14:50:09,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 15:00:08,1,trailing,front,left,up-shear,shear-perpendicular
2005-11-13 15:10:23,1,trailing,front,trailing,up-shear,shear-perpendicular
2005-11-13 15:20:09,1,trailing,front,trailing,up-shear,shear-perpendicular
2005-11-13 15:30:09,1,trailing,front,trailing,up-shear,down-shear


In [9]:
if __name__ == "__main__":

    figure_options = visualize.option.horizontal_attribute_options(
        "mcs_velocity_analysis", style="paper"
    )
    start_time = np.datetime64("2005-11-13T14:00")
    end_time = np.datetime64("2005-11-13T16:00")
    visualize.attribute.mcs_series(output_directory, start_time, end_time, figure_options, parallel=False)

2024-10-05 15:15:53,676 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.140000.nc
2024-10-05 15:15:53,713 - thor.data.aura - INFO - Creating new geographic grid with spacing 0.025 m, 0.025 m.
2024-10-05 15:15:54,112 - thor.visualize.horizontal - INFO - No figsize provided. Using default.
2024-10-05 15:15:54,200 - thor.visualize.horizontal - DEBUG - Plotting boundary.
2024-10-05 15:15:54,209 - thor.visualize.horizontal - DEBUG - Plotting boundary.
2024-10-05 15:15:54,239 - thor.visualize.attribute - DEBUG - Saving mcs_velocity_analysis figure for 2005-11-13T14:00:09.000000000.
2024-10-05 15:15:55,765 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.141000.nc
2024-10-05 15:15:55,777 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.142000.nc
2024-10-05 15:15:55,796 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.143000.nc
2024-10-05 15:15:55,821 - thor.data.aur